In [1]:
from importlib import reload
import numpy as np
import folium
import src.localization as localization
reload(localization)
from src.localization import read_data
from src.localization import plot_coeffs
from src.localization import get_distance_in_meters
from src.localization import gen_regressors
from src.localization import gen_fingerprints
from src.localization import cell_search

1) Read Data
==

In [2]:
positions, df_points = read_data('resources/data/LocTreino_Equipe_2.csv')

In [3]:
bts_positions, df_btss = read_data('resources/data/dados_BTSs.csv')

## 1.1) Plot Original Points


In [4]:
# Calculate map center
center_lat = np.mean([lat for lat in bts_positions[:, 0]])
center_lon = np.mean([lon for lon in bts_positions[:, 1]])
original_pts_map = folium.Map(location=[center_lat, center_lon],
                              zoom_start=15,
                              tiles='CartoDB dark_matter')

In [5]:
# Plot points
for location in positions:
    marker = folium.CircleMarker(location=location,
                                 radius=1,
                                 color='blue',
                                 fill_opacity=0.1)
    marker.add_to(original_pts_map)

# Plot BTSs locations
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(original_pts_map)

# Export map
original_pts_map.save("resources/maps/original_pts_map.html")

In [6]:
# Show map
original_pts_map

## Fit KNN Model

In [7]:
from sklearn import neighbors

In [10]:
# Choose columns to keep in X 
# (remove unimportant/target columns)
columns = [c for c in df_points.columns if c not in ['pontoId',
                                                     'lat',
                                                     'lon']]

# X
samples = df_points[columns].values

# Y_lat
target_lat = df_points['lat'].values
# Y_lon
target_lon = df_points['lon'].values

In [11]:
lat_reg = neighbors.KNeighborsRegressor(n_neighbors=1)
lon_reg = neighbors.KNeighborsRegressor(n_neighbors=1)

In [12]:
lat_reg.fit(samples, target_lat)
lon_reg.fit(samples, target_lon)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=1, p=2,
          weights='uniform')

In [13]:
predicted_lat = lat_reg.predict(samples)
predicted_lon = lon_reg.predict(samples)

In [16]:
# from folium.plugins import MarkerCluster
folium_map = folium.Map(location=[center_lat, center_lon],
                        zoom_start=13,
                        tiles='CartoDB dark_matter')
# cluster = MarkerCluster(list(zip(predicted_lat, predicted_lon))).add_to(folium_map)
predicted_positions = list(zip(predicted_lat, predicted_lon))
for location in predicted_positions:
    marker = folium.CircleMarker(location=location,
                                 color='red',
                                 weight=1,
                                 radius=3)
    marker.add_to(folium_map)
for location in positions:
    marker = folium.CircleMarker(location=location,
                                 color='blue',
                                 weight=1,
                                 radius=5)
    marker.add_to(folium_map)
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(folium_map)
for p1, p2 in list(zip(positions, predicted_positions)):
    folium.PolyLine([p1, p2],
                    color="green",
                    weight=1,
                    opacity=0.5).add_to(folium_map)
folium_map.save("resources/maps/knn_result_map.html")

In [17]:
from sklearn.model_selection import cross_validate

In [18]:
cv_results = cross_validate(lat_reg,
                            samples,
                            target_lat,
                            cv=10,
                            return_train_score=False)

In [19]:
cv_results['test_score']

array([0.95623138, 0.94955856, 0.95109055, 0.96697754, 0.95777126,
       0.93641567, 0.95880014, 0.93203664, 0.96924516, 0.96055802])

### Calculate Errors

In [20]:
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

def rmse(y_pred, y_real):
    return sqrt(mse(y_pred, y_real))

# Predict
predicted_lat = lat_reg.predict(samples)
predicted_lon = lon_reg.predict(samples)

# Calculate error
lat_err = rmse(predicted_lat, target_lat)
lon_err = rmse(predicted_lon, target_lon)
print('Latitude RMSE (meters):', get_distance_in_meters(0, 0, lat_err, 0))
print('Longitude RMSE (meters):', get_distance_in_meters(0, 0, 0, lon_err))

Latitude RMSE (meters): 0.0
Longitude RMSE (meters): 0.0


### Generate error map (testing with the same training points)

In [21]:
result_map = folium.Map(location=[center_lat, center_lon],
                        zoom_start=13,
                        tiles='CartoDB dark_matter')
n_points = 10000
predicted_positions = list(zip(predicted_lat, predicted_lon))
for location in predicted_positions:
    marker = folium.CircleMarker(location=location, color='red', weight=1, radius=3)
    marker.add_to(result_map)
for location in positions:
    marker = folium.CircleMarker(location=location, color='blue', weight=1, radius=3)
    marker.add_to(result_map)
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(result_map)
for p1, p2 in list(zip(positions, predicted_positions)):
    folium.PolyLine([p1, p2], color="green", weight=1, opacity=0.5).add_to(result_map)
result_map.save("resources/maps/result_map.html")